<a href="https://colab.research.google.com/github/DavidP0011/etl/blob/main/etl_mkt_02st_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INICIALIZACIÓN

In [18]:
# @title INSTALACIÓN DE LIBRERÍAS
!pip install rapidfuzz
!pip install googletrans==4.0.0-rc1


In [19]:
# @title IMPORTACIÓN DE LIBRERÍAS SISTEMA

import pandas as pd
import sys
import importlib

from google.colab import auth
auth.authenticate_user()
print('[google.colab authenticate_user]:\nAuthenticated!\n')

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Agregando la ruta a las funciones personalizadas en el path del sistema
path = '/content/drive/MyDrive/ANIMUM DIRECCION/DIRECCION BI/NOTEBOOKS'
sys.path.append(path)

# Eliminar cualquier instancia previa del módulo en la memoria
if 'dpm_SQL' in sys.modules:
    del sys.modules['dpm_SQL']

# Importar y forzar la recarga del módulo con importlib
import dpm_SQL
importlib.reload(dpm_SQL)

# Importar las funciones desde el módulo actualizado
from dpm_SQL import (
    GBQ_execute_SQL,
    map_elements_dynamic_df,
    GSheet_to_df,
    fields_name_format,
    SQL_generate_cleaning_str,
    SQL_generate_academic_date_str,
    SQL_generate_join_tables_str,
    SQL_generate_CPL_to_contacts_str,
    SQL_generate_deal_ordinal_str,
    SQL_generate_BI_view_str
)

print("Módulo dpm_SQL actualizado correctamente.")


[google.colab authenticate_user]:
Authenticated!

Mounted at /content/drive
Módulo dpm_SQL actualizado correctamente.


## GBQ DATASETS SCHEMA

In [20]:
# @title GBQ INFO GLOBAL
%%time

# Configuración
params_dic = {
    "spreadsheet_id": "https://docs.google.com/spreadsheets/d/1aJCGTJtDu_ODqBc4zUcrpQ-q6PE_HN0rO4mwYMIhCXw",
    "worksheet_name": "DATA",
    "json_keyfile": "/content/drive/Othercomputers/Mi PC/proyectos/api_keys/animum-dev-datawarehouse-ef58845eb41a.json",  # Solo necesario en Colab
}

full_info_from_GBQ_df = GSheet_to_df(params_dic)

display(full_info_from_GBQ_df)


🔄 Entorno detectado: Colab (usando JSON de credenciales)
✅ Datos extraídos con éxito de 'DATA'.


,project_id,dataset_id,table_name,field_name,field_type,num_rows,num_columns,size_mb
0,animum-dev-datawarehouse,BOMojo_staging_01,WorldwideBoxOffice_01,Year,INTEGER,5200,8,73
1,animum-dev-datawarehouse,BOMojo_staging_01,WorldwideBoxOffice_01,Rank,INTEGER,5200,8,73
2,animum-dev-datawarehouse,BOMojo_staging_01,WorldwideBoxOffice_01,Title,STRING,5200,8,73
3,animum-dev-datawarehouse,BOMojo_staging_01,WorldwideBoxOffice_01,Worldwide_Gross,INTEGER,5200,8,73
4,animum-dev-datawarehouse,BOMojo_staging_01,WorldwideBoxOffice_01,Domestic_Gross,INTEGER,5200,8,73
...,...,...,...,...,...,...,...,...
10104,animum-dev-datawarehouse,vl_01raw_01,MA_PROVINCIAS,Hora_creacion,TIMESTAMP,1029,10,3
10105,animum-dev-datawarehouse,vl_01raw_01,MA_PROVINCIAS,Usuario_creacion,STRING,1029,10,3
10106,animum-dev-datawarehouse,vl_01raw_01,MA_PROVINCIAS,Fecha_modificacion,TIMESTAMP,1029,10,3
10107,animum-dev-datawarehouse,vl_01raw_01,MA_PROVINCIAS,Hora_modificacion,TIMESTAMP,1029,10,3


CPU times: user 329 ms, sys: 18 ms, total: 347 ms
Wall time: 891 ms


In [21]:
# @title GBQ DATASETS
print("Datasets disponibles:\n")
for dataset in full_info_from_GBQ_df['dataset_id'].unique():
    print(f"{dataset}")


Datasets disponibles:

BOMojo_staging_01
FAds_staging_v01
GAds_staging_v01
IMDb_raw_01
IMDb_staging_01
cd2_01raw_01
facebook_ads_raw_v01
facebook_ads_raw_v01_facebook_ads
facebook_ads_raw_v01_facebook_ads_source
fivetran_metadata
fivetran_metadata_fivetran_platform
fivetran_metadata_stg_fivetran_platform
google_ads_raw_01
google_ads_raw_01_google_ads
google_ads_raw_01_google_ads_source
hubspot_BI_v01
hubspot_raw_v01
hubspot_raw_v01_hubspot
hubspot_raw_v01_stg_hubspot
hubspot_staging_v01
mkt_02st_01
tablas_mapeo
tp_02st_01
tp_03bi_01
vl_01raw_01


# SELECCIÓN DE TABLAS Y FIELDS

## FACEBOOK ADS

In [22]:
# @title fads_fields_selected_list})

fads_fields_selected_list = [
"date",
"country",
"impressions",
"inline_link_clicks",
"cost_per_inline_link_click",
"cpc",
"cpm",
"ctr",
"spend",
"inline_link_click_ctr"
]
fads_fields_mapped_df = pd.DataFrame({"Campo Original": fads_fields_selected_list, "Campo Formateado": fads_fields_selected_list})

In [23]:
# @title mapeo_country
mapeo_country = """
CASE
  WHEN country = 'AD' THEN 'Andorra'
  WHEN country = 'AE' THEN 'Emiratos Árabes Unidos'
  WHEN country = 'AF' THEN 'Afganistán'
  WHEN country = 'AG' THEN 'Antigua y Barbuda'
  WHEN country = 'AI' THEN 'Anguila'
  WHEN country = 'AL' THEN 'Albania'
  WHEN country = 'AM' THEN 'Armenia'
  WHEN country = 'AO' THEN 'Angola'
  WHEN country = 'AQ' THEN 'Antártida'
  WHEN country = 'AR' THEN 'Argentina'
  WHEN country = 'AS' THEN 'Samoa Americana'
  WHEN country = 'AT' THEN 'Austria'
  WHEN country = 'AU' THEN 'Australia'
  WHEN country = 'AW' THEN 'Aruba'
  WHEN country = 'AX' THEN 'Islas Åland'
  WHEN country = 'AZ' THEN 'Azerbaiyán'
  WHEN country = 'BA' THEN 'Bosnia y Herzegovina'
  WHEN country = 'BB' THEN 'Barbados'
  WHEN country = 'BD' THEN 'Bangladés'
  WHEN country = 'BE' THEN 'Bélgica'
  WHEN country = 'BF' THEN 'Burkina Faso'
  WHEN country = 'BG' THEN 'Bulgaria'
  WHEN country = 'BH' THEN 'Baréin'
  WHEN country = 'BI' THEN 'Burundi'
  WHEN country = 'BJ' THEN 'Benín'
  WHEN country = 'BL' THEN 'San Bartolomé'
  WHEN country = 'BM' THEN 'Bermudas'
  WHEN country = 'BN' THEN 'Brunéi'
  WHEN country = 'BO' THEN 'Bolivia'
  WHEN country = 'BQ' THEN 'Caribe Neerlandés'
  WHEN country = 'BR' THEN 'Brasil'
  WHEN country = 'BS' THEN 'Bahamas'
  WHEN country = 'BT' THEN 'Bután'
  WHEN country = 'BV' THEN 'Isla Bouvet'
  WHEN country = 'BW' THEN 'Botsuana'
  WHEN country = 'BY' THEN 'Bielorrusia'
  WHEN country = 'BZ' THEN 'Belice'
  WHEN country = 'CA' THEN 'Canadá'
  WHEN country = 'CC' THEN 'Islas Cocos (Keeling)'
  WHEN country = 'CD' THEN 'Rep. Dem. del Congo'
  WHEN country = 'CF' THEN 'Rep. Centroafricana'
  WHEN country = 'CG' THEN 'República del Congo'
  WHEN country = 'CH' THEN 'Suiza'
  WHEN country = 'CI' THEN 'Costa de Marfil'
  WHEN country = 'CK' THEN 'Islas Cook'
  WHEN country = 'CL' THEN 'Chile'
  WHEN country = 'CM' THEN 'Camerún'
  WHEN country = 'CN' THEN 'China'
  WHEN country = 'CO' THEN 'Colombia'
  WHEN country = 'CR' THEN 'Costa Rica'
  WHEN country = 'CU' THEN 'Cuba'
  WHEN country = 'CV' THEN 'Cabo Verde'
  WHEN country = 'CW' THEN 'Curazao'
  WHEN country = 'CX' THEN 'Isla de Navidad'
  WHEN country = 'CY' THEN 'Chipre'
  WHEN country = 'CZ' THEN 'Chequia'
  WHEN country = 'DE' THEN 'Alemania'
  WHEN country = 'DJ' THEN 'Yibuti'
  WHEN country = 'DK' THEN 'Dinamarca'
  WHEN country = 'DM' THEN 'Dominica'
  WHEN country = 'DO' THEN 'República Dominicana'
  WHEN country = 'DZ' THEN 'Argelia'
  WHEN country = 'EC' THEN 'Ecuador'
  WHEN country = 'EE' THEN 'Estonia'
  WHEN country = 'EG' THEN 'Egipto'
  WHEN country = 'EH' THEN 'Sáhara Occidental'
  WHEN country = 'ER' THEN 'Eritrea'
  WHEN country = 'ES' THEN 'España'
  WHEN country = 'ET' THEN 'Etiopía'
  WHEN country = 'FI' THEN 'Finlandia'
  WHEN country = 'FJ' THEN 'Fiyi'
  WHEN country = 'FK' THEN 'Islas Malvinas'
  WHEN country = 'FM' THEN 'Micronesia'
  WHEN country = 'FO' THEN 'Islas Feroe'
  WHEN country = 'FR' THEN 'Francia'
  WHEN country = 'GA' THEN 'Gabón'
  WHEN country = 'GB' THEN 'Reino Unido'
  WHEN country = 'GD' THEN 'Granada'
  WHEN country = 'GE' THEN 'Georgia'
  WHEN country = 'GF' THEN 'Guayana Francesa'
  WHEN country = 'GG' THEN 'Guernsey'
  WHEN country = 'GH' THEN 'Ghana'
  WHEN country = 'GI' THEN 'Gibraltar'
  WHEN country = 'GL' THEN 'Groenlandia'
  WHEN country = 'GM' THEN 'Gambia'
  WHEN country = 'GN' THEN 'Guinea'
  WHEN country = 'GP' THEN 'Guadalupe'
  WHEN country = 'GQ' THEN 'Guinea Ecuatorial'
  WHEN country = 'GR' THEN 'Grecia'
  WHEN country = 'GS' THEN 'Islas Georgias del Sur'
  WHEN country = 'GT' THEN 'Guatemala'
  WHEN country = 'GU' THEN 'Guam'
  WHEN country = 'GW' THEN 'Guinea-Bisáu'
  WHEN country = 'GY' THEN 'Guyana'
  WHEN country = 'HK' THEN 'Hong Kong'
  WHEN country = 'HM' THEN 'Islas Heard y McDonald'
  WHEN country = 'HN' THEN 'Honduras'
  WHEN country = 'HR' THEN 'Croacia'
  WHEN country = 'HT' THEN 'Haití'
  WHEN country = 'HU' THEN 'Hungría'
  WHEN country = 'ID' THEN 'Indonesia'
  WHEN country = 'IE' THEN 'Irlanda'
  WHEN country = 'IL' THEN 'Israel'
  WHEN country = 'IM' THEN 'Isla de Man'
  WHEN country = 'IN' THEN 'India'
  WHEN country = 'IO' THEN 'Terr. Británico Oc. Índico'
  WHEN country = 'IQ' THEN 'Iraq'
  WHEN country = 'IR' THEN 'Irán'
  WHEN country = 'IS' THEN 'Islandia'
  WHEN country = 'IT' THEN 'Italia'
  WHEN country = 'JE' THEN 'Jersey'
  WHEN country = 'JM' THEN 'Jamaica'
  WHEN country = 'JO' THEN 'Jordania'
  WHEN country = 'JP' THEN 'Japón'
  WHEN country = 'KE' THEN 'Kenia'
  WHEN country = 'KG' THEN 'Kirguistán'
  WHEN country = 'KH' THEN 'Camboya'
  WHEN country = 'KI' THEN 'Kiribati'
  WHEN country = 'KM' THEN 'Comoras'
  WHEN country = 'KN' THEN 'San Cristóbal y Nieves'
  WHEN country = 'KP' THEN 'Corea del Norte'
  WHEN country = 'KR' THEN 'Corea del Sur'
  WHEN country = 'KW' THEN 'Kuwait'
  WHEN country = 'KY' THEN 'Islas Caimán'
  WHEN country = 'KZ' THEN 'Kazajistán'
  WHEN country = 'LA' THEN 'Laos'
  WHEN country = 'LB' THEN 'Líbano'
  WHEN country = 'LC' THEN 'Santa Lucía'
  WHEN country = 'LI' THEN 'Liechtenstein'
  WHEN country = 'LK' THEN 'Sri Lanka'
  WHEN country = 'LR' THEN 'Liberia'
  WHEN country = 'LS' THEN 'Lesoto'
  WHEN country = 'LT' THEN 'Lituania'
  WHEN country = 'LU' THEN 'Luxemburgo'
  WHEN country = 'LV' THEN 'Letonia'
  WHEN country = 'LY' THEN 'Libia'
  WHEN country = 'MA' THEN 'Marruecos'
  WHEN country = 'MC' THEN 'Mónaco'
  WHEN country = 'MD' THEN 'Moldavia'
  WHEN country = 'ME' THEN 'Montenegro'
  WHEN country = 'MF' THEN 'San Martín (Francia)'
  WHEN country = 'MG' THEN 'Madagascar'
  WHEN country = 'MH' THEN 'Islas Marshall'
  WHEN country = 'MK' THEN 'Macedonia del Norte'
  WHEN country = 'ML' THEN 'Malí'
  WHEN country = 'MM' THEN 'Birmania (Myanmar)'
  WHEN country = 'MN' THEN 'Mongolia'
  WHEN country = 'MO' THEN 'Macao'
  WHEN country = 'MP' THEN 'Islas Marianas del Norte'
  WHEN country = 'MQ' THEN 'Martinica'
  WHEN country = 'MR' THEN 'Mauritania'
  WHEN country = 'MS' THEN 'Montserrat'
  WHEN country = 'MT' THEN 'Malta'
  WHEN country = 'MU' THEN 'Mauricio'
  WHEN country = 'MV' THEN 'Maldivas'
  WHEN country = 'MW' THEN 'Malaui'
  WHEN country = 'MX' THEN 'México'
  WHEN country = 'MY' THEN 'Malasia'
  WHEN country = 'MZ' THEN 'Mozambique'
  WHEN country = 'NA' THEN 'Namibia'
  WHEN country = 'NC' THEN 'Nueva Caledonia'
  WHEN country = 'NE' THEN 'Níger'
  WHEN country = 'NF' THEN 'Isla Norfolk'
  WHEN country = 'NG' THEN 'Nigeria'
  WHEN country = 'NI' THEN 'Nicaragua'
  WHEN country = 'NL' THEN 'Países Bajos'
  WHEN country = 'NO' THEN 'Noruega'
  WHEN country = 'NP' THEN 'Nepal'
  WHEN country = 'NR' THEN 'Nauru'
  WHEN country = 'NU' THEN 'Niue'
  WHEN country = 'NZ' THEN 'Nueva Zelanda'
  WHEN country = 'OM' THEN 'Omán'
  WHEN country = 'PA' THEN 'Panamá'
  WHEN country = 'PE' THEN 'Perú'
  WHEN country = 'PF' THEN 'Polinesia Francesa'
  WHEN country = 'PG' THEN 'Papúa Nueva Guinea'
  WHEN country = 'PH' THEN 'Filipinas'
  WHEN country = 'PK' THEN 'Pakistán'
  WHEN country = 'PL' THEN 'Polonia'
  WHEN country = 'PM' THEN 'San Pedro y Miquelón'
  WHEN country = 'PN' THEN 'Islas Pitcairn'
  WHEN country = 'PR' THEN 'Puerto Rico'
  WHEN country = 'PS' THEN 'Palestina'
  WHEN country = 'PT' THEN 'Portugal'
  WHEN country = 'PW' THEN 'Palaos'
  WHEN country = 'PY' THEN 'Paraguay'
  WHEN country = 'QA' THEN 'Catar'
  WHEN country = 'RE' THEN 'Reunión'
  WHEN country = 'RO' THEN 'Rumanía'
  WHEN country = 'RS' THEN 'Serbia'
  WHEN country = 'RU' THEN 'Rusia'
  WHEN country = 'RW' THEN 'Ruanda'
  WHEN country = 'SA' THEN 'Arabia Saudí'
  WHEN country = 'SB' THEN 'Islas Salomón'
  WHEN country = 'SC' THEN 'Seychelles'
  WHEN country = 'SD' THEN 'Sudán'
  WHEN country = 'SE' THEN 'Suecia'
  WHEN country = 'SG' THEN 'Singapur'
  WHEN country = 'SH' THEN 'Santa Elena'
  WHEN country = 'SI' THEN 'Eslovenia'
  WHEN country = 'SJ' THEN 'Islas Svalbard y Jan Mayen'
  WHEN country = 'SK' THEN 'Eslovaquia'
  WHEN country = 'SL' THEN 'Sierra Leona'
  WHEN country = 'SM' THEN 'San Marino'
  WHEN country = 'SN' THEN 'Senegal'
  WHEN country = 'SO' THEN 'Somalia'
  WHEN country = 'SR' THEN 'Surinam'
  WHEN country = 'SS' THEN 'Sudán del Sur'
  WHEN country = 'ST' THEN 'Santo Tomé y Príncipe'
  WHEN country = 'SV' THEN 'El Salvador'
  WHEN country = 'SX' THEN 'Sint Maarten'
  WHEN country = 'SY' THEN 'Siria'
  WHEN country = 'SZ' THEN 'Esuatini'
  WHEN country = 'TC' THEN 'Islas Turcas y Caicos'
  WHEN country = 'TD' THEN 'Chad'
  WHEN country = 'TF' THEN 'Territorios Australes Franceses'
  WHEN country = 'TG' THEN 'Togo'
  WHEN country = 'TH' THEN 'Tailandia'
  WHEN country = 'TJ' THEN 'Tayikistán'
  WHEN country = 'TK' THEN 'Tokelau'
  WHEN country = 'TL' THEN 'Timor Oriental'
  WHEN country = 'TM' THEN 'Turkmenistán'
  WHEN country = 'TN' THEN 'Túnez'
  WHEN country = 'TO' THEN 'Tonga'
  WHEN country = 'TR' THEN 'Turquía'
  WHEN country = 'TT' THEN 'Trinidad y Tobago'
  WHEN country = 'TV' THEN 'Tuvalu'
  WHEN country = 'TW' THEN 'Taiwán'
  WHEN country = 'TZ' THEN 'Tanzania'
  WHEN country = 'UA' THEN 'Ucrania'
  WHEN country = 'UG' THEN 'Uganda'
  WHEN country = 'UM' THEN 'Islas menores alejadas de EE.UU.'
  WHEN country = 'US' THEN 'Estados Unidos'
  WHEN country = 'UY' THEN 'Uruguay'
  WHEN country = 'UZ' THEN 'Uzbekistán'
  WHEN country = 'VA' THEN 'Ciudad del Vaticano'
  WHEN country = 'VC' THEN 'San Vicente y las Granadinas'
  WHEN country = 'VE' THEN 'Venezuela'
  WHEN country = 'VG' THEN 'Islas Vírgenes Británicas'
  WHEN country = 'VI' THEN 'Islas Vírgenes de EE.UU.'
  WHEN country = 'VN' THEN 'Vietnam'
  WHEN country = 'VU' THEN 'Vanuatu'
  WHEN country = 'WF' THEN 'Wallis y Futuna'
  WHEN country = 'WS' THEN 'Samoa'
  WHEN country = 'YE' THEN 'Yemen'
  WHEN country = 'YT' THEN 'Mayotte'
  WHEN country = 'ZA' THEN 'Sudáfrica'
  WHEN country = 'ZM' THEN 'Zambia'
  WHEN country = 'ZW' THEN 'Zimbabue'
  ELSE country
END
"""

## GOOGLE ADS

In [24]:
# @title gads_campaign_report_fields_selected_list

gads_campaign_report_fields_selected_list = [
"date_day",
"campaign_name",
"campaign_id",
"advertising_channel_type",
"advertising_channel_subtype",
"status",
"spend",
"clicks",
"impressions",
"conversions",
"conversions_value"
]
gads_campaign_report_fields_mapped_df = pd.DataFrame({"Campo Original": gads_campaign_report_fields_selected_list, "Campo Formateado": gads_campaign_report_fields_selected_list})

In [25]:
# @title gads_campaign_criterion_history_fields_selected_list

gads_campaign_criterion_history_fields_selected_list = [
"campaign_id",
"bid_modifier",
"geo_target_constant_id",
"negative",
"status",
"updated_at"
]
gads_campaign_criterion_history_fields_mapped_df = pd.DataFrame({"Campo Original": gads_campaign_criterion_history_fields_selected_list, "Campo Formateado": gads_campaign_criterion_history_fields_selected_list})

In [26]:
# @title gads_geo_target_fields_selected_list

# Listado manual
gads_geo_target_fields_selected_list = [
"id",
"canonical_name",
"name",
"target_type"
]
gads_geo_target_fields_mapped_df = pd.DataFrame({"Campo Original": gads_geo_target_fields_selected_list, "Campo Formateado": gads_geo_target_fields_selected_list})

## HUBSPOT

In [27]:
# @title hs_contact_fields_selected_list

hs_contact_fields_selected_list = [
"id",
"_fivetran_deleted",
"_fivetran_synced",
"property_a_o",
"property_address",
"property_autonomo_fecha_de_alta",
"property_autonomo_porcentaje_retencion_irpf",
"property_categorizacion_tutor_otros",
"property_city",
"property_closedate",
"property_comentarios_tutor_coordinador",
"property_comentarios_tutor_recruiter",
"property_correos_de_marketing_tasa_apertura",
"property_correos_de_marketing_tasa_de_clic",
"property_country",
"property_createdate",
"property_date_of_birth",
"property_days_to_close",
"property_dia",
"property_disponibilidad_a_colaborar",
"property_ecvt",
"property_ecvt_control",
"property_ecvt_coordinador",
"property_email",
"property_email_tutor_zoho",
"property_email_tutor_zoho_estado",
"property_es_potencial_tutor",
"property_fecha_de_nacimiento",
"property_fecha_de_nacimiento_operar",
"property_fecha_para_volver_a_valorar_al_candidato",
"property_firstname",
"property_gender",
"property_honorario_de_tutor",
"property_honorarios_cqc",
"property_hs_analytics_average_page_views",
"property_hs_analytics_first_timestamp",
"property_hs_analytics_first_visit_timestamp",
"property_hs_analytics_last_timestamp",
"property_hs_analytics_last_visit_timestamp",
"property_hs_analytics_num_page_views",
"property_hs_analytics_source",
"property_hs_email_click",
"property_hs_email_delivered",
"property_hs_last_sales_activity_timestamp",
"property_hs_last_sales_activity_type",
"property_hs_latest_meeting_activity",
"property_hs_latest_sequence_ended_date",
"property_hs_latest_sequence_enrolled",
"property_hs_latest_sequence_enrolled_date",
"property_hs_latest_sequence_finished_date",
"property_hs_latest_sequence_unenrolled_date",
"property_hs_latest_source",
"property_hs_object_id",
"property_ip_city",
"property_ip_country",
"property_ip_country_code",
"property_ip_de_pais_para_informe",
"property_ip_state",
"property_ip_state_code",
"property_lastname",
"property_num_associated_deals",
"property_phone",
"property_primera_conversion_formulario_fuente",
"property_profesion_actual",
"property_profesion_actual_rango",
"property_profesion_anterior_no_relacionada",
"property_profesion_anterior_no_relacionada_listado",
"property_profesion_anterior_relacionada",
"property_profesion_anterior_relacionada_listado",
"property_tiempo_de_dedicacion",
"property_tiempo_de_dedicacion_listado",
"property_utm_medium",
"property_nombre_fiscal",
"property_provincia",
"property_state",
"property_propuesta_2_tipo_sesion_en_directo",
"property_profesion_anterior_rol_clonado_"
]
hs_contact_fields_mapped_df = pd.DataFrame({"Campo Original": hs_contact_fields_selected_list, "Campo Formateado": hs_contact_fields_selected_list})

In [28]:
# @title hs_deal_fields_selected_list

hs_deal_fields_selected_list =[
"deal_id",
"_fivetran_deleted",
"_fivetran_synced",
"deal_pipeline_id",
"deal_pipeline_stage_id",
"is_deleted",
"owner_id",
"property_amount",
"property_ano_academico",
"property_ano_convocatoria",
"property_area",
"property_closed_lost_reason",
"property_closedate",
"property_convocatoria",
"property_correos_de_marketing_entregados_al_abrir_negocio",
"property_correos_de_marketing_entregados_al_cerrar_negocio",
"property_correos_de_marketing_entregados_al_cierre_negocio",
"property_correos_de_marketing_entregados_al_crear_negocio",
"property_correos_de_marketing_tasa_de_apertura_al_abrir_negocio",
"property_correos_de_marketing_tasa_de_apertura_al_cerrar_negocio",
"property_correos_de_marketing_tasa_de_apertura_al_cierre_negocio",
"property_correos_de_marketing_tasa_de_apertura_al_crear_negocio",
"property_correos_de_marketing_tasa_de_clic_al_abrir_negocio",
"property_correos_de_marketing_tasa_de_clic_al_cerrar_negocio",
"property_correos_de_marketing_tasa_de_clic_al_cierre_negocio",
"property_correos_de_marketing_tasa_de_clic_al_crear_negocio",
"property_createdate",
"property_days_to_close",
"property_dealname",
"property_disciplina_del_programa",
"property_engagements_last_meeting_booked",
"property_hs_days_to_close_raw",
"property_hs_was_imported",
"property_modulo_abandono",
"property_modulo_inicio",
"property_modulos_del_programa",
"property_nombre",
"property_notas_historico",
"property_notes_last_contacted",
"property_notes_last_updated",
"property_numero_de_vistas_de_pagina_al_abrir_negocio",
"property_numero_de_vistas_de_pagina_al_cerrar_negocio",
"property_numero_de_vistas_de_pagina_blog_al_abrir_negocio",
"property_numero_de_vistas_de_pagina_blog_al_cerrar_negocio",
"property_origen_de_alta_manual",
"property_puntuacion_cdm_al_abrir_negocio",
"property_puntuacion_cdm_al_cerrar_negocio",
"property_puntuacion_cdm_al_cierre_negocio",
"property_puntuacion_cdm_al_crear_negocio",
"property_solicitud_estado",
"property_solicitud_valoracion",
"property_tipo_de_beca",
"property_tipo_de_convalidacion",
"property_tipo_de_matricula",
"property_valor_cobrado",
"property_valor_contrato_matricula",
"property_valor_mermado"
]
hs_deal_fields_mapped_df = pd.DataFrame({"Campo Original": hs_deal_fields_selected_list, "Campo Formateado": hs_deal_fields_selected_list})

In [29]:
# @title hs_deal_pipeline_fields_selected_list

hs_deal_pipeline_fields_selected_list = [
"pipeline_id",
"label"
]
hs_deal_pipeline_fields_mapped_df = pd.DataFrame({"Campo Original": hs_deal_pipeline_fields_selected_list, "Campo Formateado": hs_deal_pipeline_fields_selected_list})

In [30]:
# @title hs_deal_pipeline_stage_fields_selected_list

hs_deal_pipeline_stage_fields_selected_list = [
"stage_id",
"pipeline_id",
"label",
"probability",
"closed_won"
]
hs_deal_pipeline_stage_fields_mapped_df = pd.DataFrame({"Campo Original": hs_deal_pipeline_stage_fields_selected_list, "Campo Formateado": hs_deal_pipeline_stage_fields_selected_list})

In [31]:
# @title hs_deal_contact_fields_selected_list

hs_deal_contact_fields_selected_list = [
"deal_id",
"contact_id",
"_fivetran_synced"
]
hs_deal_contact_fields_mapped_df = pd.DataFrame({"Campo Original": hs_deal_contact_fields_selected_list, "Campo Formateado": hs_deal_contact_fields_selected_list})

# SQL SCRIPT TABLES GENERATION

## PARÁMETROS GENERATIVOS

In [32]:
# @title ** GENERALES

# @markdown **::..**
# @markdown ___
# @markdown > # Generales
##########################################################################################
# Comunes
exclude_records_by_creation_date_bool = False                                    # @param {"type":"boolean"}
exclude_records_by_creation_date_field = "date_day"                              # @param {"type":"string"}
exclude_records_by_creation_date_range = {"from": "2021-06-01"}                 # @param {"type":"string"}

exclude_records_by_keywords_bool = False                                         # @param {"type":"boolean"}
exclude_records_by_keywords_fields = [ ]                                        # Lista vacía es todo # @param {"type":"string"}
exclude_records_by_keywords_words = keywords_descartar = [     "test",     "prueba",     "pruebas",     "ejemplo",     "demo",     "temporal",     "dummy",     "asdf",     "asd",     "lorem",     "ipsum",     "aaa",     "bbbb",     "cccc",     "xxxx",     "zzz",     "null",     "nulo",     "vacio",     "sin datos",     "pendiente",     "eliminar",     "descartar" ]   # @param
fields_to_trim = [ ]                                                            # Lista vacía es todo # @param

# from ipywidgets import interact
# def f(x):
#     return x * 2
# interact(f, x=10)



## FACEBOOK ADS

In [36]:
# @title fads_cleaned

config = {
    "table_source":           "animum-dev-datawarehouse.facebook_ads_raw_v01.demographics_country",
    "table_destination":      "animum-dev-datawarehouse.mkt_02st_01.fads_demographics_country_cleaned",
    "fields_destination_prefix": "fads_dc_",
    "fields_mapped_use": False,
    "fields_mapped_df": fads_fields_mapped_df,

    "exclude_records_by_creation_date_bool":  exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field":  exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords":            exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields":      exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words":      exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}


print(SQL_generate_cleaning_str(config))

params = {
    'GCP_project_id': 'animum-dev-datawarehouse',
    'SQL_script': SQL_generate_cleaning_str(config)
}

GBQ_execute_SQL(params)

CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.fads_demographics_country_cleaned` AS
SELECT
  `date` AS `fads_dc_date`,
  `country` AS `fads_dc_country`,
  `impressions` AS `fads_dc_impressions`,
  `inline_link_clicks` AS `fads_dc_inline_link_clicks`,
  `cost_per_inline_link_click` AS `fads_dc_cost_per_inline_link_click`,
  `cpc` AS `fads_dc_cpc`,
  `cpm` AS `fads_dc_cpm`,
  `ctr` AS `fads_dc_ctr`,
  `spend` AS `fads_dc_spend`,
  `inline_link_click_ctr` AS `fads_dc_inline_link_click_ctr`
FROM `animum-dev-datawarehouse.facebook_ads_raw_v01.demographics_country`
WHERE TRUE
;
[INFO]: Iniciando el cliente de BigQuery...
[INFO]: Cliente de BigQuery inicializado en el proyecto: animum-dev-datawarehouse

[INFO]: Acción detectada en el script SQL: CREATE
[INFO]: Resumen del script SQL:
CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.fads_demographics_country_cleaned` AS
SELECT
  `date` AS `fads_dc_date`,
  `country` AS `fads_dc_country`,
  `impressions` AS `fads_dc

In [46]:
# @title fads_cleaned_country_mapped

SQL = """
UPDATE `animum-dev-datawarehouse.mkt_02st_01.fads_demographics_country_cleaned`
SET fads_dc_country = CASE
  WHEN fads_dc_country = 'AD' THEN 'Andorra'
  WHEN fads_dc_country = 'AE' THEN 'Emiratos Árabes Unidos'
  WHEN fads_dc_country = 'AF' THEN 'Afganistán'
  WHEN fads_dc_country = 'AG' THEN 'Antigua y Barbuda'
  WHEN fads_dc_country = 'AI' THEN 'Anguila'
  WHEN fads_dc_country = 'AL' THEN 'Albania'
  WHEN fads_dc_country = 'AM' THEN 'Armenia'
  WHEN fads_dc_country = 'AO' THEN 'Angola'
  WHEN fads_dc_country = 'AR' THEN 'Argentina'
  WHEN fads_dc_country = 'AT' THEN 'Austria'
  WHEN fads_dc_country = 'AU' THEN 'Australia'
  WHEN fads_dc_country = 'AW' THEN 'Aruba'
  WHEN fads_dc_country = 'AZ' THEN 'Azerbaiyán'
  WHEN fads_dc_country = 'BA' THEN 'Bosnia y Herzegovina'
  WHEN fads_dc_country = 'BB' THEN 'Barbados'
  WHEN fads_dc_country = 'BD' THEN 'Bangladés'
  WHEN fads_dc_country = 'BE' THEN 'Bélgica'
  WHEN fads_dc_country = 'BF' THEN 'Burkina Faso'
  WHEN fads_dc_country = 'BG' THEN 'Bulgaria'
  WHEN fads_dc_country = 'BH' THEN 'Baréin'
  WHEN fads_dc_country = 'BI' THEN 'Burundi'
  WHEN fads_dc_country = 'BJ' THEN 'Benín'
  WHEN fads_dc_country = 'BM' THEN 'Bermudas'
  WHEN fads_dc_country = 'BN' THEN 'Brunéi'
  WHEN fads_dc_country = 'BO' THEN 'Bolivia'
  WHEN fads_dc_country = 'BR' THEN 'Brasil'
  WHEN fads_dc_country = 'BS' THEN 'Bahamas'
  WHEN fads_dc_country = 'BT' THEN 'Bután'
  WHEN fads_dc_country = 'BW' THEN 'Botsuana'
  WHEN fads_dc_country = 'BY' THEN 'Bielorrusia'
  WHEN fads_dc_country = 'BZ' THEN 'Belice'
  WHEN fads_dc_country = 'CA' THEN 'Canadá'
  WHEN fads_dc_country = 'CH' THEN 'Suiza'
  WHEN fads_dc_country = 'CL' THEN 'Chile'
  WHEN fads_dc_country = 'CM' THEN 'Camerún'
  WHEN fads_dc_country = 'CN' THEN 'China'
  WHEN fads_dc_country = 'CO' THEN 'Colombia'
  WHEN fads_dc_country = 'CR' THEN 'Costa Rica'
  WHEN fads_dc_country = 'CU' THEN 'Cuba'
  WHEN fads_dc_country = 'CY' THEN 'Chipre'
  WHEN fads_dc_country = 'CZ' THEN 'Chequia'
  WHEN fads_dc_country = 'DE' THEN 'Alemania'
  WHEN fads_dc_country = 'DK' THEN 'Dinamarca'
  WHEN fads_dc_country = 'DO' THEN 'República Dominicana'
  WHEN fads_dc_country = 'DZ' THEN 'Argelia'
  WHEN fads_dc_country = 'EC' THEN 'Ecuador'
  WHEN fads_dc_country = 'EE' THEN 'Estonia'
  WHEN fads_dc_country = 'EG' THEN 'Egipto'
  WHEN fads_dc_country = 'ES' THEN 'España'
  WHEN fads_dc_country = 'ET' THEN 'Etiopía'
  WHEN fads_dc_country = 'FI' THEN 'Finlandia'
  WHEN fads_dc_country = 'FJ' THEN 'Fiyi'
  WHEN fads_dc_country = 'FR' THEN 'Francia'
  WHEN fads_dc_country = 'GA' THEN 'Gabón'
  WHEN fads_dc_country = 'GB' THEN 'Reino Unido'
  WHEN fads_dc_country = 'GD' THEN 'Granada'
  WHEN fads_dc_country = 'GE' THEN 'Georgia'
  WHEN fads_dc_country = 'GH' THEN 'Ghana'
  WHEN fads_dc_country = 'GI' THEN 'Gibraltar'
  WHEN fads_dc_country = 'GL' THEN 'Groenlandia'
  WHEN fads_dc_country = 'GM' THEN 'Gambia'
  WHEN fads_dc_country = 'GN' THEN 'Guinea'
  WHEN fads_dc_country = 'GR' THEN 'Grecia'
  WHEN fads_dc_country = 'GT' THEN 'Guatemala'
  WHEN fads_dc_country = 'GU' THEN 'Guam'
  WHEN fads_dc_country = 'GY' THEN 'Guyana'
  WHEN fads_dc_country = 'HK' THEN 'Hong Kong'
  WHEN fads_dc_country = 'HN' THEN 'Honduras'
  WHEN fads_dc_country = 'HR' THEN 'Croacia'
  WHEN fads_dc_country = 'HT' THEN 'Haití'
  WHEN fads_dc_country = 'HU' THEN 'Hungría'
  WHEN fads_dc_country = 'ID' THEN 'Indonesia'
  WHEN fads_dc_country = 'IE' THEN 'Irlanda'
  WHEN fads_dc_country = 'IL' THEN 'Israel'
  WHEN fads_dc_country = 'IN' THEN 'India'
  WHEN fads_dc_country = 'IQ' THEN 'Iraq'
  WHEN fads_dc_country = 'IR' THEN 'Irán'
  WHEN fads_dc_country = 'IS' THEN 'Islandia'
  WHEN fads_dc_country = 'IT' THEN 'Italia'
  WHEN fads_dc_country = 'JP' THEN 'Japón'
  WHEN fads_dc_country = 'KE' THEN 'Kenia'
  WHEN fads_dc_country = 'KR' THEN 'Corea del Sur'
  WHEN fads_dc_country = 'MX' THEN 'México'
  WHEN fads_dc_country = 'US' THEN 'Estados Unidos'
  WHEN fads_dc_country = 'VE' THEN 'Venezuela'
  ELSE fads_dc_country
END
WHERE fads_dc_country IN (
  'AD', 'AE', 'AF', 'AG', 'AI', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AW', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG',
  'BH', 'BI', 'BJ', 'BM', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CH', 'CL', 'CM', 'CN', 'CO', 'CR',
  'CU', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'ET', 'FI', 'FJ', 'FR', 'GA', 'GB', 'GD', 'GE',
  'GH', 'GI', 'GL', 'GM', 'GN', 'GR', 'GT', 'GU', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ',
  'IR', 'IS', 'IT', 'JP', 'KE', 'KR', 'MX', 'US', 'VE'
);
"""

params = {
    'GCP_project_id': 'animum-dev-datawarehouse',
    'SQL_script': SQL
}

GBQ_execute_SQL(params)

[INFO]: Iniciando el cliente de BigQuery...
[INFO]: Cliente de BigQuery inicializado en el proyecto: animum-dev-datawarehouse

[INFO]: Acción detectada en el script SQL: UPDATE
[INFO]: Resumen del script SQL:
UPDATE `animum-dev-datawarehouse.mkt_02st_01.fads_demographics_country_cleaned`
SET fads_dc_country = CASE
  WHEN fads_dc_country = 'AD' THEN 'Andorra'
  WHEN fads_dc_country = 'AE' THEN 'Emiratos Árabes Unidos'
  WHEN fads_dc_country = 'AF' THEN 'Afganistán'
...
[INFO]: Ejecutando el script SQL...
[INFO]: Tiempo de inicio: 2025-02-13 11:30:26 UTC
[SUCCESS]: Consulta SQL ejecutada exitosamente.

[INFO]: Detalles del trabajo:
- ID del trabajo: 309030ae-4058-4900-9945-8f2202071397
- Estado del trabajo: DONE
- Tiempo de creación: 2025-02-13 11:30:27.060000+00:00
- Bytes procesados: 453857
- Filas afectadas reportadas: 0
- Filas insertadas en la tabla de destino: 2867
[INFO]: Tiempo total de ejecución: 3.21 segundos

[INFO]: Tamaño actual de la tabla destino: 44.80 KB


## GOOGLE ADS

In [37]:
# @title gads_campaign_report_cleaned

config = {
    "table_source":           "animum-dev-datawarehouse.google_ads_raw_01_google_ads.google_ads__campaign_report",
    "table_destination":      "animum-dev-datawarehouse.mkt_02st_01.gads_campaign_report_cleaned",
    "fields_destination_prefix": "gads_r_",
    "fields_mapped_use": False,
    "fields_mapped_df": gads_campaign_report_fields_mapped_df,

    "exclude_records_by_creation_date_bool":  exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field":  exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords":            exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields":      exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words":      exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}

print(SQL_generate_cleaning_str(config))

params = {
    'GCP_project_id': 'animum-dev-datawarehouse',
    'SQL_script': SQL_generate_cleaning_str(config)
}

GBQ_execute_SQL(params)


CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.gads_campaign_report_cleaned` AS
SELECT
  `date_day` AS `gads_r_date_day`,
  `campaign_name` AS `gads_r_campaign_name`,
  `campaign_id` AS `gads_r_campaign_id`,
  `advertising_channel_type` AS `gads_r_advertising_channel_type`,
  `advertising_channel_subtype` AS `gads_r_advertising_channel_subtype`,
  `status` AS `gads_r_status`,
  `spend` AS `gads_r_spend`,
  `clicks` AS `gads_r_clicks`,
  `impressions` AS `gads_r_impressions`,
  `conversions` AS `gads_r_conversions`,
  `conversions_value` AS `gads_r_conversions_value`
FROM `animum-dev-datawarehouse.google_ads_raw_01_google_ads.google_ads__campaign_report`
WHERE TRUE
;
[INFO]: Iniciando el cliente de BigQuery...
[INFO]: Cliente de BigQuery inicializado en el proyecto: animum-dev-datawarehouse

[INFO]: Acción detectada en el script SQL: CREATE
[INFO]: Resumen del script SQL:
CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.gads_campaign_report_cleaned` AS
SELECT

In [38]:
# @title gads_campaign_criterion_history_cleaned

config = {
    "table_source":           "animum-dev-datawarehouse.google_ads_raw_01.campaign_criterion_history",
    "table_destination":      "animum-dev-datawarehouse.mkt_02st_01.gads_campaign_criterion_history_cleaned",
    "fields_destination_prefix": "gads_ch_",
    "fields_mapped_use": False,
    "fields_mapped_df": gads_campaign_criterion_history_fields_mapped_df,

    "exclude_records_by_creation_date_bool":  exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field":  exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords":            exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields":      exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words":      exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}

print(SQL_generate_cleaning_str(config))

params = {
    'GCP_project_id': 'animum-dev-datawarehouse',
    'SQL_script': SQL_generate_cleaning_str(config)
}

GBQ_execute_SQL(params)


CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.gads_campaign_criterion_history_cleaned` AS
SELECT
  `campaign_id` AS `gads_ch_campaign_id`,
  `bid_modifier` AS `gads_ch_bid_modifier`,
  `geo_target_constant_id` AS `gads_ch_geo_target_constant_id`,
  `negative` AS `gads_ch_negative`,
  `status` AS `gads_ch_status`,
  `updated_at` AS `gads_ch_updated_at`
FROM `animum-dev-datawarehouse.google_ads_raw_01.campaign_criterion_history`
WHERE TRUE
;
[INFO]: Iniciando el cliente de BigQuery...
[INFO]: Cliente de BigQuery inicializado en el proyecto: animum-dev-datawarehouse

[INFO]: Acción detectada en el script SQL: CREATE
[INFO]: Resumen del script SQL:
CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.gads_campaign_criterion_history_cleaned` AS
SELECT
  `campaign_id` AS `gads_ch_campaign_id`,
  `bid_modifier` AS `gads_ch_bid_modifier`,
  `geo_target_constant_id` AS `gads_ch_geo_target_constant_id`,
...
[INFO]: Ejecutando el script SQL...
[INFO]: Tiempo de inicio: 20

In [39]:
# @title gads_geo_target_cleaned

config = {
    "table_source":           "animum-dev-datawarehouse.google_ads_raw_01.geo_target",
    "table_destination":      "animum-dev-datawarehouse.mkt_02st_01.gads_geo_target_cleaned",
    "fields_destination_prefix": "gads_gt_",
    "fields_mapped_use": False,
    "fields_mapped_df": gads_geo_target_fields_mapped_df,

    "exclude_records_by_creation_date_bool":  exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field":  exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords":            exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields":      exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words":      exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}

print(SQL_generate_cleaning_str(config))

params = {
    'GCP_project_id': 'animum-dev-datawarehouse',
    'SQL_script': SQL_generate_cleaning_str(config)
}

GBQ_execute_SQL(params)

CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.gads_geo_target_cleaned` AS
SELECT
  `id` AS `gads_gt_id`,
  `canonical_name` AS `gads_gt_canonical_name`,
  `name` AS `gads_gt_name`,
  `target_type` AS `gads_gt_target_type`
FROM `animum-dev-datawarehouse.google_ads_raw_01.geo_target`
WHERE TRUE
;
[INFO]: Iniciando el cliente de BigQuery...
[INFO]: Cliente de BigQuery inicializado en el proyecto: animum-dev-datawarehouse

[INFO]: Acción detectada en el script SQL: CREATE
[INFO]: Resumen del script SQL:
CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.gads_geo_target_cleaned` AS
SELECT
  `id` AS `gads_gt_id`,
  `canonical_name` AS `gads_gt_canonical_name`,
  `name` AS `gads_gt_name`,
...
[INFO]: Ejecutando el script SQL...
[INFO]: Tiempo de inicio: 2025-02-13 11:10:29 UTC
[SUCCESS]: Consulta SQL ejecutada exitosamente.

[INFO]: Detalles del trabajo:
- ID del trabajo: 26af1d0f-5c4e-4387-b285-b8c6d1fc3c3c
- Estado del trabajo: DONE
- Tiempo de creación: 2025-02-1

## HUBSPOT

In [40]:
# @title hs_contact_cleaned

config = {
    "table_source":           "animum-dev-datawarehouse.hubspot_raw_v01.contact",
    "table_destination":      "animum-dev-datawarehouse.hs_02st_01.hs_contact_cleaned",
    "fields_destination_prefix": "hs_c_",
    "fields_mapped_use": False,
    "fields_mapped_df": hs_contact_fields_mapped_df,

    "exclude_records_by_creation_date_bool": exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field": exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords_bool": exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields": exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words": exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}

# Generar y mostrar la consulta SQL
print(SQL_generate_cleaning_str(config))

params = {
    'GCP_project_id': 'animum-dev-datawarehouse',
    'SQL_script': SQL_generate_cleaning_str(config)
}

GBQ_execute_SQL(params)

CREATE OR REPLACE TABLE `animum-dev-datawarehouse.hs_02st_01.hs_contact_cleaned` AS
SELECT
  `id` AS `hs_c_id`,
  `_fivetran_deleted` AS `hs_c__fivetran_deleted`,
  `_fivetran_synced` AS `hs_c__fivetran_synced`,
  `property_a_o` AS `hs_c_property_a_o`,
  `property_address` AS `hs_c_property_address`,
  `property_autonomo_fecha_de_alta` AS `hs_c_property_autonomo_fecha_de_alta`,
  `property_autonomo_porcentaje_retencion_irpf` AS `hs_c_property_autonomo_porcentaje_retencion_irpf`,
  `property_categorizacion_tutor_otros` AS `hs_c_property_categorizacion_tutor_otros`,
  `property_city` AS `hs_c_property_city`,
  `property_closedate` AS `hs_c_property_closedate`,
  `property_comentarios_tutor_coordinador` AS `hs_c_property_comentarios_tutor_coordinador`,
  `property_comentarios_tutor_recruiter` AS `hs_c_property_comentarios_tutor_recruiter`,
  `property_correos_de_marketing_tasa_apertura` AS `hs_c_property_correos_de_marketing_tasa_apertura`,
  `property_correos_de_marketing_tasa_de_clic` A

In [41]:
# @title hs_deal_cleaned

config = {
    "table_source":           "animum-dev-datawarehouse.hubspot_raw_v01.deal",
    "table_destination":      "animum-dev-datawarehouse.mkt_02st_01.hs_deal_cleaned",
    "fields_destination_prefix": "hs_d_",
    "fields_mapped_use": False,
    "fields_mapped_df": hs_deal_fields_mapped_df,

    "exclude_records_by_creation_date_bool":  exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field":  exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords":            exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields":      exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words":      exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}

print(SQL_generate_cleaning_str(config))

params = {
    'GCP_project_id': 'animum-dev-datawarehouse',
    'SQL_script': SQL_generate_cleaning_str(config)
}

GBQ_execute_SQL(params)

CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.hs_deal_cleaned` AS
SELECT
  `deal_id` AS `hs_d_deal_id`,
  `_fivetran_deleted` AS `hs_d__fivetran_deleted`,
  `_fivetran_synced` AS `hs_d__fivetran_synced`,
  `deal_pipeline_id` AS `hs_d_deal_pipeline_id`,
  `deal_pipeline_stage_id` AS `hs_d_deal_pipeline_stage_id`,
  `is_deleted` AS `hs_d_is_deleted`,
  `owner_id` AS `hs_d_owner_id`,
  `property_amount` AS `hs_d_property_amount`,
  `property_ano_academico` AS `hs_d_property_ano_academico`,
  `property_ano_convocatoria` AS `hs_d_property_ano_convocatoria`,
  `property_area` AS `hs_d_property_area`,
  `property_closed_lost_reason` AS `hs_d_property_closed_lost_reason`,
  `property_closedate` AS `hs_d_property_closedate`,
  `property_convocatoria` AS `hs_d_property_convocatoria`,
  `property_correos_de_marketing_entregados_al_abrir_negocio` AS `hs_d_property_correos_de_marketing_entregados_al_abrir_negocio`,
  `property_correos_de_marketing_entregados_al_cerrar_negocio` AS `h

In [42]:
# @title hs_deal_pipeline_cleaned

config = {
    "table_source":           "animum-dev-datawarehouse.hubspot_raw_v01.deal_pipeline",
    "table_destination":      "animum-dev-datawarehouse.mkt_02st_01.hs_deal_pipeline_cleaned",
    "fields_destination_prefix": "hs_dp_",
    "fields_mapped_use": False,
    "fields_mapped_df": hs_deal_pipeline_fields_mapped_df,

    "exclude_records_by_creation_date_bool":  exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field":  exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords":            exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields":      exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words":      exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}

print(SQL_generate_cleaning_str(config))

params = {
    'GCP_project_id': 'animum-dev-datawarehouse',
    'SQL_script': SQL_generate_cleaning_str(config)
}

GBQ_execute_SQL(params)

CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.hs_deal_pipeline_cleaned` AS
SELECT
  `pipeline_id` AS `hs_dp_pipeline_id`,
  `label` AS `hs_dp_label`
FROM `animum-dev-datawarehouse.hubspot_raw_v01.deal_pipeline`
WHERE TRUE
;
[INFO]: Iniciando el cliente de BigQuery...
[INFO]: Cliente de BigQuery inicializado en el proyecto: animum-dev-datawarehouse

[INFO]: Acción detectada en el script SQL: CREATE
[INFO]: Resumen del script SQL:
CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.hs_deal_pipeline_cleaned` AS
SELECT
  `pipeline_id` AS `hs_dp_pipeline_id`,
  `label` AS `hs_dp_label`
FROM `animum-dev-datawarehouse.hubspot_raw_v01.deal_pipeline`
...
[INFO]: Ejecutando el script SQL...
[INFO]: Tiempo de inicio: 2025-02-13 11:10:40 UTC
[SUCCESS]: Consulta SQL ejecutada exitosamente.

[INFO]: Detalles del trabajo:
- ID del trabajo: 31a04d55-7d43-4e50-9a5b-70d0e040fa3b
- Estado del trabajo: DONE
- Tiempo de creación: 2025-02-13 11:10:40.743000+00:00
- Bytes procesados

In [43]:
# @title hs_deal_pipeline_stage_cleaned

config = {
    "table_source":           "animum-dev-datawarehouse.hubspot_raw_v01.deal_pipeline_stage",
    "table_destination":      "animum-dev-datawarehouse.mkt_02st_01.hs_deal_pipeline_stage_cleaned",
    "fields_destination_prefix": "hs_dps_",
    "fields_mapped_use": False,
    "fields_mapped_df": hs_deal_pipeline_stage_fields_mapped_df,

    "exclude_records_by_creation_date_bool":  exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field":  exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords":            exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields":      exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words":      exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}

print(SQL_generate_cleaning_str(config))

params = {
    'GCP_project_id': 'animum-dev-datawarehouse',
    'SQL_script': SQL_generate_cleaning_str(config)
}

GBQ_execute_SQL(params)

CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.hs_deal_pipeline_stage_cleaned` AS
SELECT
  `stage_id` AS `hs_dps_stage_id`,
  `pipeline_id` AS `hs_dps_pipeline_id`,
  `label` AS `hs_dps_label`,
  `probability` AS `hs_dps_probability`,
  `closed_won` AS `hs_dps_closed_won`
FROM `animum-dev-datawarehouse.hubspot_raw_v01.deal_pipeline_stage`
WHERE TRUE
;
[INFO]: Iniciando el cliente de BigQuery...
[INFO]: Cliente de BigQuery inicializado en el proyecto: animum-dev-datawarehouse

[INFO]: Acción detectada en el script SQL: CREATE
[INFO]: Resumen del script SQL:
CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.hs_deal_pipeline_stage_cleaned` AS
SELECT
  `stage_id` AS `hs_dps_stage_id`,
  `pipeline_id` AS `hs_dps_pipeline_id`,
  `label` AS `hs_dps_label`,
...
[INFO]: Ejecutando el script SQL...
[INFO]: Tiempo de inicio: 2025-02-13 11:10:43 UTC
[SUCCESS]: Consulta SQL ejecutada exitosamente.

[INFO]: Detalles del trabajo:
- ID del trabajo: 217135df-d73c-44b5-8e89-55

In [44]:
# @title hs_deal_contact_cleaned

config = {
    "table_source":         "animum-dev-datawarehouse.hubspot_raw_v01.deal_contact",
    "table_destination":    "animum-dev-datawarehouse.mkt_02st_01.hs_deal_contact_cleaned",
    "fields_destination_prefix": "hs_dc_",
    "fields_mapped_use":   False,
    "fields_mapped_df":    hs_deal_contact_fields_mapped_df,


    "exclude_records_by_creation_date_bool":  exclude_records_by_creation_date_bool,
    "exclude_records_by_creation_date_field":  exclude_records_by_creation_date_field,
    "exclude_records_by_creation_date_range": exclude_records_by_creation_date_range,

    "exclude_records_by_keywords":            exclude_records_by_keywords_bool,
    "exclude_records_by_keywords_fields":      exclude_records_by_keywords_fields,
    "exclude_records_by_keywords_words":      exclude_records_by_keywords_words,

    "fields_to_trim": fields_to_trim
}

print(SQL_generate_cleaning_str(config))

params = {
    'GCP_project_id': 'animum-dev-datawarehouse',
    'SQL_script': SQL_generate_cleaning_str(config)
}

GBQ_execute_SQL(params)

CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.hs_deal_contact_cleaned` AS
SELECT
  `deal_id` AS `hs_dc_deal_id`,
  `contact_id` AS `hs_dc_contact_id`,
  `_fivetran_synced` AS `hs_dc__fivetran_synced`
FROM `animum-dev-datawarehouse.hubspot_raw_v01.deal_contact`
WHERE TRUE
;
[INFO]: Iniciando el cliente de BigQuery...
[INFO]: Cliente de BigQuery inicializado en el proyecto: animum-dev-datawarehouse

[INFO]: Acción detectada en el script SQL: CREATE
[INFO]: Resumen del script SQL:
CREATE OR REPLACE TABLE `animum-dev-datawarehouse.mkt_02st_01.hs_deal_contact_cleaned` AS
SELECT
  `deal_id` AS `hs_dc_deal_id`,
  `contact_id` AS `hs_dc_contact_id`,
  `_fivetran_synced` AS `hs_dc__fivetran_synced`
...
[INFO]: Ejecutando el script SQL...
[INFO]: Tiempo de inicio: 2025-02-13 11:10:47 UTC
[SUCCESS]: Consulta SQL ejecutada exitosamente.

[INFO]: Detalles del trabajo:
- ID del trabajo: 1f47c12a-f6f0-4a53-b5a5-782ff212f2d3
- Estado del trabajo: DONE
- Tiempo de creación: 2025-02-13 1